# **Test `CodeRepresentation`**

## Environment

In [ ]:
!python --version

## Dependencies

In [ ]:
!rm *.*

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install tokenizers
!pip install transformers

# Run

In [ ]:
!ls -al *.sh

In [ ]:
!cat run_code_repr_calculation_colab_test.sh

In [ ]:
!bash run_code_repr_calculation_colab_test.sh

In [ ]:
!ls -la

In [ ]:
!python -u code_representation.py --install --download --data pytest --gpu --repr cb --verbose 2 --limit 10

# Debug

In [ ]:
import torch
import numpy as np

from code_representation import CodeRepresentation

In [ ]:
 cr = CodeRepresentation(install=True, download=True, data='pytest', model_name='microsoft/graphcodebert-base', device_id='cuda:0', verbose=1)

In [ ]:
corpora = cr.create_corpora()

In [ ]:
 tree_sitter_lib = cr.build_tree_sitter_lib()
 parser = cr.create_parser(tree_sitter_lib, lang='python')

In [ ]:
def run(sources, padding=True, truncation=True):
    with torch.no_grad():
        tokens_pt = cr.representator._tokenizer(sources, padding=padding, truncation=truncation, return_tensors='pt')
        input_ids = tokens_pt['input_ids']
        attention_mask = tokens_pt['attention_mask']

        output = cr.representator._pipe.model.roberta(input_ids=input_ids.to(cr.representator._device),
                                                      attention_mask=attention_mask.to(cr.representator._device))
        hidden_states = output.last_hidden_state
        attention_mask.unsqueeze_(-1)
        attention_mask_big = attention_mask.expand(attention_mask.shape[0], attention_mask.shape[1], hidden_states.shape[2]).to(cr.representator._device)
        masked_summa = torch.sum(hidden_states * attention_mask_big, dim=1)

        embeddings = masked_summa / torch.sum(attention_mask_big, dim=1)
        bounds = torch.count_nonzero(attention_mask, dim=1)

        return embeddings.detach().cpu().numpy().astype(np.half), bounds.detach().cpu().numpy()